In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
import pandas as pd
from nba_predict import *
import json
import ast

In [21]:
with open('F:/Documents/_DataScience/nba_data/test.json', 'r') as f:
    data = json.loads(f.read())
    
raw_df = pd.DataFrame(data)   

In [63]:
raw_df.columns

Index(['box_score', 'day', 'home_city', 'home_line', 'home_name', 'summary',
       'vis_city', 'vis_line', 'vis_name'],
      dtype='object')

In [121]:
def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    return ast.literal_eval(d)

def list_of_dicts(ld):
    '''
    Create a mapping of the tuples formed after 
    converting json strings of list to a python list   
    '''
    return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

def extract_json_column(df, col, prefix):
    '''
    Convert a column containing json objects to a new df
    '''
    return json_normalize(df[col].astype("str").apply(only_dict).tolist()).add_prefix(prefix)

def fix_column_names(x):
    '''
    Tidy up column names from file for easier reading
    '''
    
    y = x.replace('-', '_')
    y = y.lower()
    y = y.replace('team_', '')
    return y

def extract_box_scores(df):
    home = extract_json_column(df, 'home_line', 'home_')
    away = extract_json_column(df, 'vis_line', 'away_')
    
    box = home.merge(away, left_index=True, right_index=True)
    box = box.merge(df[['day']], left_index=True, right_index=True)
    
    box.rename(fix_column_names, axis='columns', inplace=True)
    box.drop(['home_city', 'away_city'], inplace=True, axis=1)
    
    #Reorder
    to_front = ['day', 'home_name', 'away_name']
    cols = scores.columns.tolist()
    for i in to_front:
        cols.remove(i)
    cols = to_front + cols
    box['day'] = pd.to_datetime(box['day'], format='%m_%d_%y')
       
    return box[cols]
        
scores = extract_box_scores(raw_df)
scores.sort_values(by=['day'], ascending=False, inplace=True)
scores.head()

,day,home_name,away_name,home_ast,home_fg3_pct,home_fg_pct,home_ft_pct,home_losses,home_pts,home_pts_qtr1,...,away_ft_pct,away_losses,away_pts,away_pts_qtr1,away_pts_qtr2,away_pts_qtr3,away_pts_qtr4,away_reb,away_tov,away_wins
1486,2017-03-26,Nets,Hawks,16,19,35,59,36,92,20,...,83,57,107,32,23,23,29,50,17,16
1312,2017-03-25,Wizards,Cavaliers,24,36,49,78,25,115,26,...,81,28,127,40,31,25,31,40,11,45
412,2017-03-25,Jazz,Clippers,14,26,45,89,30,108,28,...,72,29,95,14,27,29,25,43,14,44
967,2017-03-22,Wizards,Hawks,23,32,40,74,28,104,16,...,83,34,100,20,26,21,33,44,16,37
737,2017-03-22,Pacers,Celtics,27,41,51,86,26,109,21,...,68,35,100,19,23,32,26,45,13,36
